In [591]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta, date, time
import numpy as np
from my_keys import *

In [590]:
# get timestamp in correct format for tomorrow 0,12 and 24 
def get_tomorrow():
    #make tomorow day, by adding one day to today
    date_tomorrow = date.today() + timedelta(days=1)
    #make three times: 00:00:00,12:00:00,23:59:59
    time_start = time(hour=0, minute=0, second=0)
    time_middle = time(hour=12, minute=0, second=0)
    time_end = time(hour=23, minute=59, second=59)
    #combine tomorow day and the times
    datetime_start = datetime.combine(date_tomorrow, time_start).strftime('%Y-%m-%dT%H:%M')
    datetime_middle = datetime.combine(date_tomorrow, time_middle).strftime('%Y-%m-%dT%H:%M')
    datetime_end = datetime.combine(date_tomorrow, time_end).strftime('%Y-%m-%dT%H:%M')
    #return three tomorow times
    return datetime_start,datetime_middle,datetime_end

In [535]:
# weather_forcast function that het a list of citie names and return the weather_forcast for tomorow
def weather_forcast(cities_df):
   
    
    cities_list  = []
    
    weather_list = []
    error =[]
    for index, city in cities_df.iterrows():
        city_r = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city['city']}&appid={my_openweather_api_key}&units=metric")
        
        if city_r.status_code != 200:
            error.append(city+" got errpr code: "+ str(city_r.status_code))
            break
        
        city_j = city_r.json()
        
        cities_list.append(city_j['city'])
        

        for detail in city_j['list']:
            weather_details_dic = {}
            
            # weather_details_dic["city_name"] = city_j['city']['name']
            weather_details_dic["temperature"] = detail['main']['temp']
            weather_details_dic["temp_min"] = detail['main']['temp_min']
            weather_details_dic["temp_max"] = detail['main']['temp_max']
            weather_details_dic["feels_like"] = detail['main']['feels_like']
            weather_details_dic["pressure"] = detail['main']['pressure']
            weather_details_dic["humidity"] = detail['main']['humidity']
            weather_details_dic["weather"] = detail['weather'][0]['main']
            weather_details_dic["weather_description"] = detail['weather'][0]['description']
            weather_details_dic["clouds"] = detail['clouds']['all']
            weather_details_dic["wind_speed"] = detail['wind']['speed']
            weather_details_dic["date_time_3h"] = detail['dt_txt']
            # weather_details_dic["city_id"] = city_j['city']['id']
            weather_details_dic["city_id"] = city['city_id']
            weather_list.append(weather_details_dic)
    
    
    df_weather = pd.DataFrame(weather_list)
    dt_start,dt_middle,dt_end = get_tomorrow()
    df_weather['date_time_3h'] = pd.to_datetime(df_weather['date_time_3h'])
    df_weather = df_weather.loc[lambda df_ : (df_['date_time_3h'] >= dt_start) &  (df_['date_time_3h'] <= dt_end)]
    
    
    df_cities = pd.json_normalize(cities_list)
    df_cities.rename(columns={'id':'city_id'}, inplace=True)
    
    return df_weather,df_cities,error

In [510]:
import re
from bs4 import BeautifulSoup


def get_cities(cities_list):

    city_columns= ['id', 'wikiDataId', 'type', 'name', 'country', 'countryCode', 'region', 'mayor', 'elevationMeters', 'latitude', 'longitude',	'population', 'timezone']
    cities_df = pd.DataFrame(columns= city_columns)
    for city in cities_list :
        wiki_url = f"https://en.wikipedia.org/wiki/{city}"

        wiki_city_r = requests.get(wiki_url)
        wiki_city_soup = BeautifulSoup(wiki_city_r.content, "html.parser")
        try: city_mayor = wiki_city_soup.find('a' , text = re.compile(".*Mayor.*")).parent.parent.find('td').find('a').string
        except: city_mayor = 'Unknown'
        #  city_wiki_id = wiki_city_soup.select('li#t-wikibase.mw-list-item a')[0]['href'].split('/')[-1]
        city_wiki_id = wiki_city_soup.select('div#mw-navigation div#mw-panel nav#p-tb div.vector-menu-content ul.vector-menu-content-list li#t-wikibase.mw-list-item a')[0]['href'].split('/')[-1]
        
        url = f"https://wft-geo-db.p.rapidapi.com/v1/geo/cities/{city_wiki_id}"

        headers = {
	                "X-RapidAPI-Key": "84c9131c0dmsh2cb633d63800cd4p15f8f9jsn1298da2c4cd0",
	                "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
                    }

        city_informatin_r = requests.request("GET", url, headers=headers)
        city_informatin_j = city_informatin_r.json()
        city_informatin_df = pd.json_normalize(city_informatin_j['data'])
        city_informatin_df["mayor"] = city_mayor
        city_informatin_df = city_informatin_df[city_columns]
        cities_df = pd.concat([cities_df,city_informatin_df])
    city_columns_rename= {'id' : 'city_id', 'wikiDataId' : 'wiki_data_id', 'name' : 'city', 'elevationMeters' : 'elevation', 'countryCode' : 'country_code'}
    cities_df.rename(columns = city_columns_rename, inplace=True)
    cities_df['population'] = pd.to_numeric(cities_df['population'])
    cities_df['city_id'] = pd.to_numeric(cities_df['city_id'])
    cities_df.reset_index(drop=True, inplace= True)
    return cities_df

In [542]:
def get_citys_airports(city_name, country_iso2, city_id):
    
	url = "https://aerodatabox.p.rapidapi.com/airports/search/term"

	querystring = {"q":city_name,"limit":"10"}

	headers = {
		"X-RapidAPI-Key": my_RapidAPI_key,
		"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
	}
	error = []
	airports_r = requests.request("GET", url, headers=headers, params=querystring)
	columns=['icao','iata','name','short_name','longitude_deg','iso_country','latitude_deg','city_name','city_id']
	empty_df = pd.DataFrame(columns=columns)
	if airports_r.status_code != 200:
		error.append(city_name + " got errpr code: "+ str(airports_r.status_code))
		return empty_df,error
    
	airports_j = airports_r.json()
	if len(airports_j['items'])==0:
		error.append(city_name + " has no airport")
		return empty_df,error
	df_airports =pd.json_normalize(airports_j['items'])
	df_airports = df_airports.loc[lambda df_ : df_['countryCode'] == country_iso2]
	df_airports.rename(columns={'location.lat' : 'latitude_deg', 'location.lon' : 'longitude_deg', 'municipalityName': 'city_name', 'countryCode' : 'iso_country',
                             'shortName' : 'short_name'}, inplace=True)
	df_airports['city_id'] = pd.to_numeric(city_id)
	df_airports = df_airports[columns]
	return df_airports,error

In [571]:
def get_arrival_flights(airports):
    
    # make api call to receive flight information
    columns=['flight_NO', 'flight_status', 'dep_airport', 'dep_airport_icao', 'dep_airport_iata','sched_arr_loc_time',
                                    'sched_arr_Utc', 'terminal' , 'airline','aircraf', 'icao']
    rename_dict= {'number':'flight_NO', 'status':'flight_status', 
                        'movement.airport.name' : 'dep_airport','movement.airport.icao' : 'dep_airport_icao', 'movement.airport.iata':'dep_airport_iata', 
                        'movement.scheduledTimeLocal' : 'sched_arr_loc_time', 'movement.scheduledTimeUtc' : 'sched_arr_Utc', 'movement.terminal': 'terminal',
                        'airline.name':'airline', 'aircraft.model' :'aircraf'}
    arrival_flights_df = pd.DataFrame(columns=columns)
    error = []
    dt_start,dt_middle,dt_end = get_tomorrow()
    
    for index,airport in airports.iterrows():
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport['icao']}/{dt_middle}/{dt_end}"
        querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"false"}
        headers = {
            "X-RapidAPI-Key": my_RapidAPI_key,
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

        arrivals_r = requests.request("GET", url, headers=headers, params=querystring)
        
        if arrivals_r.status_code != 200:
            error.append(airport['icao'] + " got errpr code: " + str(arrivals_r.status_code))
            continue
        
        arrivals_j = arrivals_r.json()
        
        if len(arrivals_j['arrivals'])==0:
            error.append(airport['icao'] + " has no flight")
            continue
        
        arrivals_df = pd.json_normalize(arrivals_j['arrivals']).sort_values(by='movement.scheduledTimeLocal')

        
        # clean df
        # arrivals_df = arrivals_df.loc[lambda df_ : df_['isCargo']== False]
        
        try: arrivals_df = arrivals_df[['number', 'status', 
                        'movement.airport.name','movement.airport.icao', 'movement.airport.iata', 
                        'movement.scheduledTimeLocal', 'movement.scheduledTimeUtc', 'movement.terminal'
                        'airline.name', 'aircraft.model']]
        except: arrivals_df = arrivals_df[['number', 'status', 
                        'movement.airport.name','movement.airport.icao', 'movement.airport.iata', 
                        'movement.scheduledTimeLocal', 'movement.scheduledTimeUtc',
                        'airline.name', 'aircraft.model']]
        
        arrivals_df.rename(columns=rename_dict , inplace=True)
        arrivals_df['icao'] = airport['icao']
    
        arrival_flights_df = pd.concat([arrival_flights_df,arrivals_df])

    return arrival_flights_df,error


In [ ]:
#PART1
cities = ['Berlin','Paris','Amsterdam','Barcelona','Rome','Lisbon','Prague','Vienna','Madrid']
cities_df = get_cities(cities)

In [530]:
#PART2
weather_data, cities_df_2 ,cities_error= weather_forcast(cities_df)

In [543]:
#PART3

airports_df = pd.DataFrame(columns=['icao','iata','name','short_name','longitude_deg','iso_country','latitude_deg','city_name','city_id'])

# airports_df = pd.concat([get_citys_airports(city['city'] ,city['country_code'],city['city_id']) for index,city in cities_df.iterrows()])

for index,city in cities_df.iterrows():
    city_airports, airports_error = get_citys_airports(city['city'] ,city['country_code'],city['city_id'])
    airports_df = pd.concat([airports_df,city_airports])
airports_df.reset_index(drop=True, inplace=True)


In [589]:
# airports_df

In [572]:
#PART4
# arrival_flights_df = pd.DataFrame(columns=['flight_NO', 'flight_status', 'dep_airport', 'dep_airport_icao', 'dep_airport_iata','sched_arr_loc_time',
                                    # 'sched_arr_Utc', 'terminal' , 'airline','aircraf', 'icao'])

# arrival_flights,flights_error = get_arrival_flights(airports_df)
# arrival_flights_df = pd.concat([arrival_flights_df,arrival_flights])
# arrival_flights_df.reset_index(drop=True, inplace=True)

arrival_flights_df,flights_error = get_arrival_flights(airports_df)

save the data in mysql

In [302]:
import pandas as pd
import sqlalchemy
import pymysql

In [584]:

con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [585]:
cities_df['elevation'] =cities_df['elevation'].fillna(value=0)
(
cities_df
    .to_sql('cities', con=con, if_exists='append', index=False)
    )

9

In [587]:
airports_df.to_sql('airports', con=con, if_exists='append', index=False)

16

In [586]:
weather_data.to_sql('weather', con=con, if_exists='append', index=False)

72

In [588]:
arrival_flights_df.fillna(value ="Unknown").to_sql('arrivals', con=con, if_exists='append', index=False)

3568